# **Отбор признаков: мотивация**

[**Отбор признаков**](https://lms.skillfactory.ru/courses/course-v1:SkillFactory+DSPR-2.0+14JULY2021/jump_to_id/68496461c02343d0a7b1fd901edd40bc) — это процесс выбора важных признаков, оказывающих наибольшее влияние на предсказание.

In [3]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

### **Загрузка данных**

In [2]:
%%capture
!wget https://www.dropbox.com/s/64ol9q9ssggz6f1/data_ford_price.xlsx

In [4]:
data = pd.read_excel('data/data_ford_price.xlsx') 

### **Предобработка данных**

Давайте оценим влияние мультиколлинеарности на линейную регрессию:

In [5]:
data = data[['price','year', 'cylinders', 'odometer', 'lat', 'long', 'weather']]
data.dropna(inplace = True)

y = data['price']
x = data.drop(columns='price')

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

### **Обучение модели**

In [6]:
model = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

MAE: 4682.957


### **Удаление избыточного признака**

Мы выяснили, что у нас присутствует сильная зависимость между lat и weather. Удалим lat, так как этот признак, в отличие от weather, необходимо округлять.

In [7]:
x.drop('lat', axis = 1, inplace = True)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

In [9]:
model = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

MAE: 4672.930


✍ Итак, мы вспомнили о понятии мультиколлинеарности и о его влиянии на отбор признаков. В следующем юните мы разберём автоматические методы отбора признаков, влияющих на качество моделирования.

#  **Отбор признаков: классификация методов**

Методы отбора признаков предназначены для уменьшения количества входных переменных до тех значений, которые наиболее полезны для предсказательной способности модели.

Фича - признак
![](https://lms.skillfactory.ru/assets/courseware/v1/a9a03b27389e7e8d9b8180f545260056/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/dst3-ml6-9_5.png)

### **МЕТОД РЕКУРСИВНОГО ИСКЛЮЧЕНИЯ ПРИЗНАКОВ (RFE)**

**Метод рекурсивного исключения признаков (RFE)** предполагает выбор признаков путём рекурсивного рассмотрения всё меньших и меньших наборов фичей.

Сначала RFE обучается на изначальной выборке, и происходит оценка важности каждого признака. Затем наименее важные фичи удаляются. Эта процедура рекурсивно повторяется на сокращённом наборе до тех пор, пока в конечном итоге не будет достигнуто желаемое количество признаков в выборке.

In [1]:
from sklearn.feature_selection import RFE

In [10]:
y = data['price']
x = data.drop(columns='price')

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

Выделим три наиболее значимых признака:

In [11]:
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=3, step=1)
selector = selector.fit(X_train, y_train)
 
selector.get_feature_names_out()

array(['year', 'cylinders', 'lat'], dtype=object)

Также узнаем, как RFE проранжировал все доступные признаки:

In [14]:
# 3 самых важных признака
print(X_train.columns)
# сила влияния признаков
print(selector.ranking_)

Index(['year', 'cylinders', 'odometer', 'lat', 'long', 'weather'], dtype='object')
[1 1 4 1 3 2]


##  **МЕТОДЫ ВЫБОРА ПРИЗНАКОВ НА ОСНОВЕ ФИЛЬТРОВ**

В качестве фильтров для выбора признаков используются уже знакомые нам статистики, такие как коэффициент **корреляции Пирсона**, **ANOVA** и т. д. При этом выбор статистических показателей сильно зависит от типов переменных в данных.

Чем больше известно о типе данных, тем проще выбрать подходящую статистическую меру для метода отбора признаков на основе фильтра. Ниже приведена схема-помощник в выборе метода селекции признаков.

Если входящий признак числовой, то если и на выходе тоже число - это задача регрессии, если же на выходе категориальный признак - то это задача классификации.

![](https://lms.skillfactory.ru/assets/courseware/v1/8f55a77ec813c98e6e48f2efff923f23/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/dst3-ml6-9_3.png)

Библиотека **sklearn** обеспечивает реализацию большинства полезных статистических показателей, например:

* коэффициента корреляции Пирсона: [**f_regression()**](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_regression.html);
* дисперсионного анализа ANOVA: [**f_classif()**](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_classif.html);
* хи-квадрата: [**chi2()**](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.chi2.html);
* взаимной информации: [**mutual_info_classif()**](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_classif.html) и [**mutual_info_regression()**](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_regression.html).

Кроме того, библиотека **SciPy** обеспечивает реализацию многих других статистических данных, таких как тау Кендалла ([**kendalltau**](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.kendalltau.html)) и ранговая корреляция Спирмена ([**spearmanr**](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.spearmanr.html)).
***
**sklearn** также предоставляет множество различных методов фильтрации после расчёта статистики для каждой входной переменной с целевой.

Два наиболее популярных метода:

* выбор k лучших переменных: [**SelectKBest**](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html);
* выбор переменных верхнего процентиля: [**SelectPercentile**](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectPercentile.html).

In [15]:
from sklearn.feature_selection import SelectKBest, f_regression

In [16]:
selector = SelectKBest(f_regression, k=3)
selector.fit(X_train, y_train)
 
selector.get_feature_names_out()

array(['year', 'cylinders', 'odometer'], dtype=object)

На этот раз odometer оказался в топе.